In [1]:
output_path = '../Data/Intermediate_Files/'

# Import functions to clean up clinical data
from FM_Functions.Clinical_Data_CleanUp import *

# Call functions to merge, index and clean clinical data files
labels_0531         = clean_cog       (merge_index_0531())
labels_1031         = clean_cog       (merge_index_1031())
labels_aml05        = clean_aml05     (merge_index_aml05())
labels_beataml      = clean_beataml   (merge_index_beataml())
labels_amltcga      = clean_amltcga   (merge_index_amltcga())
labels_nordic_all   = clean_nordic_all(merge_index_nordic_all())
labels_mds_taml     = clean_mds_taml  (merge_index_mds_taml())
labels_all_graal    = clean_all_graal (merge_index_all_graal())
labels_target_all   = clean_target_all(merge_index_target_all())

# Combine all clinical data labels into one dataframe
labels_combined = pd.concat([labels_aml05, labels_beataml,
                        labels_0531, labels_amltcga, labels_1031,
                        labels_nordic_all, labels_mds_taml,
                        labels_all_graal,labels_target_all], axis=0, join='outer')

# Read df
df = pd.read_pickle(output_path + 'df_batch_corrected.pkl')

# Remove samples that are not in the methyl dataset
df_labels = labels_combined.loc[labels_combined.index.isin(df.index)].sort_index()

# Save the clinical data labels
df_labels.to_csv(output_path + 'clinical_data.csv')

print('The clinical data has been indexed and cleaned.\n\
Exclusion of samples may be applied depending on the analysis.')

The clinical data has been indexed and cleaned.
Exclusion of samples may be applied depending on the analysis.


In [5]:
def classify_controls(normal_samples):
    mapping = {
        'Bone Marrow Normal': 'Otherwise-Normal Control',
        'Blood Derived Normal': 'Otherwise-Normal Control'}
    
    for key, value in mapping.items():
        if key in normal_samples:
            return value


def classify_fusion(gene_fusion):
    mapping = {
    'RUNX1-RUNX1T1': 'AML with t(8;21)(q22;q22.1)/RUNX1::RUNX1T1',
    'CBFB-MYH11':    'AML with inv(16)(p13.1q22) or t(16;16)(p13.1;q22)/CBFB::MYH11',
    'KMT2A':         'AML with t(9;11)(p22;q23.3)/KMT2A-rearrangement',
    'add(11)(q23)':  'AML with t(9;11)(p22;q23.3)/KMT2A-rearrangement',
    'PML-RARA':      'APL with t(15;17)(q24.1;q21.2)/PML::RARA',
    'DEK-NUP214':    'AML with t(6;9)(p23;q34.1)/DEK::NUP214',
    'MECOM':         'AML with inv(3)(q21.3q26.2) or t(3;3)(q21.3;q26.2)/MECOM-rearrangement',
    'PRDM16-RPN1':   'AML with other rare recurring translocations',
    'RBM15-MRTFA':   'AML with other rare recurring translocations',
    'RBM15-MKL1':    'AML with other rare recurring translocations',
    'NPM1-MLF1':     'AML with other rare recurring translocations',
    'NUP98-NSD1':    'AML with other rare recurring translocations',
    'ETV6':          'AML with other rare recurring translocations',
    'KAT6A-CREBBP':  'AML with other rare recurring translocations',
    'MLLT10':        'AML with other rare recurring translocations',
    'PICALM':        'AML with other rare recurring translocations',
    'NUP98':         'AML with other rare recurring translocations',
    'PICALM-MLLT10': 'AML with other rare recurring translocations',
    'FUS-ERG':       'AML with other rare recurring translocations',
    'RUNX1-CBFA2T3': 'AML with other rare recurring translocations',
    'CBFA2T3-GLIS2': 'AML with other rare recurring translocations',
    'CBFA2T3-GLIS3': 'AML with other rare recurring translocations',
    'PSIP1-NUP214':  'AML with other rare recurring translocations',
    'XPO1-TNRC18':   'AML with other rare recurring translocations', 
    'HNRNPH1-ERG':   'AML with other rare recurring translocations',
    'NIPBL-HOXB9':   'AML with other rare recurring translocations', 
    'SET-NUP214':    'AML with other rare recurring translocations', 
    'FLI1-IFIT2':    'AML with other rare recurring translocations', 
    'TCF4-ZEB2':     'AML with other rare recurring translocations',
    'MBTD1-ZMYND11': 'AML with other rare recurring translocations', 
    'FOSB-KLF6':     'AML with other rare recurring translocations', 
    'SFPQ-ZFP36L2':  'AML with other rare recurring translocations', 
    'RUNX1-LINC00478':'AML with other rare recurring translocations',
    'RUNX1-EVX1':     'AML with other rare recurring translocations', 
    'NPM1-RARA':      'AML with other rare recurring translocations', 
    'PSPC1-ZFP36L1':  'AML with other rare recurring translocations', 
    'EWSR1-FEV':      'AML with other rare recurring translocations',
    'STAG2-AFF2(deletion)': 'AML with other rare recurring translocations', 
    'MYB-GATA1':            'AML with other rare recurring translocations', 
    'CBFA2T3-GLIS3':        'AML with other rare recurring translocations',
    'RUNX1-ZFPM2':          'AML with other rare recurring translocations', 
    'NPM1-CCDC28A':         'AML with other rare recurring translocations',
    'RUNX1-CBFA2T2':        'AML with other rare recurring translocations',
    'PIM3-BRD1':            'AML with other rare recurring translocations',
    'KAT6A-EP300':          'AML with other rare recurring translocations',
    'DOT1L-RPS15':          'AML with other rare recurring translocations',
    'FUS-FEV':              'AML with other rare recurring translocations',
    'KAT6A-NCOA2':          'AML with other rare recurring translocations',
    'other MLL':            'AML with other rare recurring translocations',
    'JARID2-PTP4A1':        'AML with other rare recurring translocations',
    'FUS-FLI1':             'AML with other rare recurring translocations',        
    'BCR-ABL1':             'AML with t(9;22)(q34.1;q11.2)/BCR::ABL1'
    }
    
    for key, value in mapping.items():
        if key in gene_fusion:
            return value

def classify_cebpa(cebpa_mutation):
    mapping = {
        'Yes': 'AML with in-frame bZIP mutated CEBPA'}
    
    for key, value in mapping.items():
        if key in cebpa_mutation:
            return value

def classify_npm(npm_mutation):
    mapping = {
        'Yes': 'AML with mutated NPM1',
    }

    for key, value in mapping.items():
        if key in npm_mutation:
            return value
        
def classify_annotated_diagnosis(diagnosis):
    mapping = {
        'mutated NPM1': 'AML with mutated NPM1',
        'mutated CEBPA': 'AML with in-frame bZIP mutated CEBPA',
        'myelodysplasia-related changes': 'AML with myelodysplasia-related changes'
        }
    
    for key, value in mapping.items():
        if key in diagnosis:
            return value

labels_0531['ELN22_Controls'] = labels_0531['Sample Type'].astype(str).apply(classify_controls)
labels_0531['ELN22_Gene Fusion'] = labels_0531['Gene Fusion'].astype(str).apply(classify_fusion)
labels_0531['ELN22_CEBPA'] = labels_0531['CEBPA mutation'].astype(str).apply(classify_cebpa)
labels_0531['ELN22_NPM1'] = labels_0531['NPM mutation'].astype(str).apply(classify_npm)
labels_0531['ELN22_Comment'] = labels_0531['Comment'].astype(str).apply(classify_annotated_diagnosis)


labels_0531['Combined Diagnosis'] = labels_0531[['ELN22_Controls','ELN22_Gene Fusion', 'ELN22_CEBPA', 'ELN22_NPM1', 'ELN22_Comment']]\
    .apply(lambda x: ','.join(filter(lambda i: i is not None and i==i, x)), axis=1)

# Replace empty strings with NaN
labels_0531['Combined Diagnosis'] = labels_0531['Combined Diagnosis'].replace('', np.nan)

# Create `ELN22 Final Diagnosis` column by splitting `Combined Diagnosis` by comma and taking the first element
labels_0531['ELN22 Final Diagnosis'] = labels_0531['Combined Diagnosis'].str.split(',').str[0]

labels_0531['ELN22 Final Diagnosis'].value_counts(dropna=False)


ELN22 Final Diagnosis
Otherwise-Normal Control                                                  181
NaN                                                                       137
AML with t(9;11)(p22;q23.3)/KMT2A-rearrangement                           120
AML with other rare recurring translocations                              105
AML with t(8;21)(q22;q22.1)/RUNX1::RUNX1T1                                 92
AML with inv(16)(p13.1q22) or t(16;16)(p13.1;q22)/CBFB::MYH11              90
AML with mutated NPM1                                                      41
AML with in-frame bZIP mutated CEBPA                                       31
AML with t(6;9)(p23;q34.1)/DEK::NUP214                                     17
AML with inv(3)(q21.3q26.2) or t(3;3)(q21.3;q26.2)/MECOM-rearrangement      2
APL with t(15;17)(q24.1;q21.2)/PML::RARA                                    1
Name: count, dtype: int64

In [3]:
labels_0531['Combined Diagnosis'].value_counts(dropna=False)

Combined Diagnosis
NaN                                                                                                137
AML with t(9;11)(p22;q23.3)/KMT2A-rearrangement                                                    117
AML with other rare recurring translocations                                                        99
Otherwise-Normal Control                                                                            94
AML with t(8;21)(q22;q22.1)/RUNX1::RUNX1T1                                                          91
AML with inv(16)(p13.1q22) or t(16;16)(p13.1;q22)/CBFB::MYH11                                       89
AML with mutated NPM1                                                                               41
Otherwise-Normal Control,AML with other rare recurring translocations                               38
AML with in-frame bZIP mutated CEBPA                                                                31
Otherwise-Normal Control,AML with t(9;11)(p22;q23.3)/K

In [6]:
labels_0531.shape

(817, 93)

In [27]:
labels_0531['Sample Type'].value_counts(dropna=False)

Sample Type
Diagnosis                                            459
Bone Marrow Normal                                   147
Primary Blood Derived Cancer - Bone Marrow            91
Recurrent Blood Derived Cancer - Bone Marrow          58
Fibroblasts from Bone Marrow Normal                   27
Primary Blood Derived Cancer - Peripheral Blood       22
Blood Derived Normal                                   7
Recurrent Blood Derived Cancer - Peripheral Blood      6
Name: count, dtype: int64

In [19]:
labels_1031[labels_1031['ELN22 Final Diagnosis'].isnull()]['Sample Type'].value_counts(dropna=False)

KeyError: 'ELN22 Final Diagnosis'

In [18]:
labels_0531[labels_0531['ELN22 Final Diagnosis'].isnull()]['Sample Type'].value_counts(dropna=False)

Sample Type
Diagnosis                                            83
Bone Marrow Normal                                   78
Primary Blood Derived Cancer - Bone Marrow           34
Fibroblasts from Bone Marrow Normal                  15
Recurrent Blood Derived Cancer - Bone Marrow         13
Primary Blood Derived Cancer - Peripheral Blood       5
Recurrent Blood Derived Cancer - Peripheral Blood     2
Blood Derived Normal                                  1
Name: count, dtype: int64

In [15]:
labels_0531.to_csv(output_path + 'temp.csv')